# 1. Multistrain proteogenomics - Identifications

In [1]:
import pandas as pd
from Bio import SeqIO
from collections import Counter, defaultdict
import pprint
import json
import numpy as np
from collections import Counter
import shutil
%load_ext autoreload
%autoreload 1
%aimport functions


In [2]:
txt_path  = 'data/txt/'
outfolder = 'output/'
analysis_folder = 'data/analysis/'

In [3]:
summary = pd.read_csv(txt_path + 'summary.txt', sep='\t')
summary[['Max. missed cleavages', 'Enzyme', 'Enzyme mode']].drop_duplicates().dropna()

,Max. missed cleavages,Enzyme,Enzyme mode
0,3.0,Trypsin/P,Specific


In [4]:
parameters = pd.read_csv(txt_path + 'parameters.txt', sep='\t')
parameters

,Parameter,Value
0,Version,1.5.3.12
1,User name,shaun
2,Machine name,BLACKBURN2
3,Date of writing,06/14/2016 02:55:56
4,Fixed modifications,Carbamidomethyl (C)
5,Decoy mode,revert
6,Special AAs,KR
7,Include contaminants,True
8,MS/MS tol. (FTMS),20 ppm
9,Top MS/MS peaks per 100 Da. (FTMS),12


# 3.1 Protein and peptide identifications 

In [5]:
print("Parameters: ")
print('PSM FDR: ',parameters.set_index('Parameter').loc['PSM FDR'].values[0])
print('Protein FDR: ',parameters.set_index('Parameter').loc['Protein FDR'].values[0])
print("\nIdentifications: ")
print('MS/MS Submitted: ', int(summary[-1:]['MS/MS Submitted'].values[0]))
print('MS/MS Identified: ', int(summary[-1:]['MS/MS Identified'].values[0]))
print('MS/MS Identified (%):', summary[-1:]['MS/MS Identified [%]'].values[0])



Parameters: 
PSM FDR:  0.01
Protein FDR:  0.01

Identifications: 
MS/MS Submitted:  818580
MS/MS Identified:  346305
MS/MS Identified (%): 42.31


In [6]:
res = functions.process_txt(txt_path, "HypoHyper", outfolder)

Analysis: HypoHyper
data/txt/
HypoHyper: Total peptides:  21834
HypoHyper: Total target peptides:  21652
HypoHyper: Total proteins:  2363
HypoHyper: Total target proteins:  2327


/cbio/users/ptgmat003/2023/Thys/mtb/manuscripts/proteogenomics/Proteogenomics-Manuscript/functions.py:253: DtypeWarning: Columns (50,51,59) have mixed types. Specify dtype option on import or set low_memory=False.
  evidences = pd.read_csv('{}/evidence.txt'.format(path), sep='\t')


In [7]:
# get peptide identification stats
identifications = pd.read_csv( analysis_folder + 'stats/identifications.csv')
index_cols = identifications.columns[:1].tolist()
identifications.set_index(index_cols, inplace=True)
identifications.index.names=[None]

identifications.to_excel('tables/table1.xlsx')
identifications

,S507,S5527,combined
identified peptides,20312,20465,21652
specific peptides,20154,20305,21484
non-genomic peptides,5,5,10
identified protein groups,2326,2326,2327
specific protein groups,2324,2323,2325
paralogous specific protein groups,20,20,20


# 3.2 Proteogenomic analysis

## 3.2.1 Protein Identifications

In [8]:
H37Rv_results = pd.read_csv(analysis_folder + '/H37Rv_combined.csv',sep='\t')
CDC1551_results = pd.read_csv(analysis_folder + '/CDC1551_combined.csv',sep='\t')

print("Number of annotated protein groups in H37Rv: ", len(H37Rv_results['Reference BLAST - all strains'].dropna()))
print("Number of annotated protein groups in CDC1551: ", len(CDC1551_results['Reference BLAST - all strains'].dropna()))
print()
H37Rv_identifications=pd.read_csv('data/analysis/annotations/export/H37Rv/Combined_Annotated_ORFs.csv')
CDC1551_identifications=pd.read_csv('data/analysis/annotations/export/CDC1551/Combined_Annotated_ORFs.csv')
print('H37Rv proteins identified: ', len(H37Rv_identifications.drop_duplicates('BLASTP')))
print('CDC1551 proteins identified:', len(CDC1551_identifications.drop_duplicates('BLASTP')))

shutil.copyfile(analysis_folder + 'annotations/export/H37Rv/pathview/mtu01100.protein_identifications.png', 'figures/figure1.png')

Number of annotated protein groups in H37Rv:  2309
Number of annotated protein groups in CDC1551:  2288

H37Rv proteins identified:  2098
CDC1551 proteins identified: 2065


'figures/figure1.png'

## 3.2.2 Differences in N-terminus acetylation identified in key virulence factors

In [9]:
all_acetylated = pd.read_csv(analysis_folder + 'annotations/export/H37Rv/Combined_Proteins_Nterm_Acetylation.csv')
print("Total acetylated proteins: " , len(all_acetylated))
res = Counter(all_acetylated['Strains'])
for key in res:
    print(key+ ':', res[key])
differences = pd.read_csv(analysis_folder + 'annotations/export/H37Rv/Combined_Proteins_Nterm_Acetylation_Differences.csv')
del differences['Unnamed: 0']
differences = differences.sort_values(['Strains'])
differences = differences.reset_index()
del differences['index']
shutil.copyfile(analysis_folder + 'annotations/export/H37Rv/pathview/mtu01100.nterm_acetyl_all.png', 'figures/figure2.png')
differences

Total acetylated proteins:  110
S507;S5527: 82
S5527: 17
S507: 11


,BLASTP,Strains,Gene Name,Protein Name
0,P9WMS7,S507,guaA,GMP synthase [glutamine-hydrolyzing]
1,P9WQD9,S507,kasA,3-oxoacyl-[acyl-carrier-protein] synthase 1
2,P9WLT1,S507,Rv1708,Uncharacterized protein Rv1708
3,P71922,S507,Rv2426c,AAA+ ATPase domain-containing protein
4,P9WK15,S507,mbtK,Lysine N-acyltransferase MbtK
5,O69727,S507,fadE36,Possible acyl-CoA dehydrogenase FadE36
6,P9WIH3,S507,pckG,Phosphoenolpyruvate carboxykinase [GTP]
7,O53842,S507,Rv0831c,Conserved protein
8,O50398,S507,Rv3369,Conserved protein
9,I6XCB2,S507,scpB,Possible segregation and condensation protein ...


In [10]:
targets = pd.read_csv(analysis_folder + 'annotations/export/H37Rv/Combined_Proteins_Nterm_Acetylation_Targets.csv')
del targets['Unnamed: 0']
targets = targets.sort_values(['Strains'])
targets  = targets.reset_index()
del targets['index']
print(Counter(targets['Strains']))
targets.to_excel('tables/table2.xlsx')
targets

Counter({'S5527': 13, 'S507': 6})


,BLASTP,Strains,Gene Name,Protein Name
0,O50398,S507,Rv3369,Conserved protein
1,O53842,S507,Rv0831c,Conserved protein
2,P9WMS7,S507,guaA,GMP synthase [glutamine-hydrolyzing]
3,P9WLT1,S507,Rv1708,Uncharacterized protein Rv1708
4,P9WQD9,S507,kasA,3-oxoacyl-[acyl-carrier-protein] synthase 1
5,P9WIH3,S507,pckG,Phosphoenolpyruvate carboxykinase [GTP]
6,O06154,S5527,Rv1637c,Conserved protein
7,P9WP79,S5527,fbiB,Bifunctional F420 biosynthesis protein FbiB
8,P9WP71,S5527,ctaD,Probable cytochrome c oxidase subunit 1
9,P9WMK7,S5527,hupB,DNA-binding protein HupB


## 3.2.3-4 Comparison of N-terminus post-processing between strains.

In [11]:
# TSSs stats here are after dropping paralogous ORFs (one ORF per cluster)
tss= pd.read_csv(analysis_folder + 'stats/tss.csv')

tss.to_excel('supplementary/s2.xlsx')
tss

,Unnamed: 0,Methionine translation of non-ATG,MAP cleaved start codon,Non-tryptic start codon,Total TSS peptides,MAP percentage,Start codon count GTG,Start codon count GTG (%),Start codon count ATG,Start codon count ATG (%),...,Nterm not acetylated MAP TSS peptides,Nterm not acetylated iMet TSS peptides,Nterm acetylated iMet TSS peptides,ORFs with 1 TSSs,ORFs with 2 TSSs,ORFs with 3 TSSs,ORFs with 4 TSSs,ORFs with 5 TSSs,ORFs with 6 TSSs,ORFs with 7 TSSs
0,S5527,100,597,157,854,69.91,320.0,37.47,419.0,49.06,...,502,249,8,614,62,15,4,4,0,1
1,S507,96,573,166,835,68.62,312.0,37.37,411.0,49.22,...,485,253,9,600,52,15,8,3,1,0


## 3.2.5 Evidence for multiple translation initiation sites found. 

In [12]:
# ORF TSS counts including paralogous ORFs, after excluding peptides that are not specific in all strains (Globally Specific Peptides)
annotations = pd.read_csv(analysis_folder + '/annotations/summary.csv')
annotations = annotations.rename(columns={"Unnamed: 0":''})
annotations.to_excel('supplementary/s3.xlsx')
annotations

,,S5527,S507,Combined
0,GloballySpecificPeptides,20300,20149,21483
1,CDC1551 Reference entries identified,2065,2063,2065
2,Annotated TSS validated - CDC1551,409,409,434
3,Annotated TSS validated (proteins) - CDC1551,409,409,434
4,Downstream TSS identified - CDC1551,325,310,397
5,Downstream TSS identified (proteins) - CDC1551,268,256,314
6,Upstream non-TSS peptide with no putative upst...,62,64,93
7,Upstream non-TSS peptide with no putative upst...,62,64,93
8,Upstream TSS identified - CDC1551,51,38,56
9,Upstream TSS identified (proteins) - CDC1551,51,38,56


In [13]:
def show_messages(peptide_annotations, orf_id):
    peptide_annotations = peptide_annotations.drop_duplicates(['ORF_id','PeptideSequence', 'Ref_id'])
    peptide_annotations = peptide_annotations[(peptide_annotations['ORF_id'] == orf_id)  & (peptide_annotations['TSS'] == True)]
    
    peptide_annotations['PeptideLength'] = peptide_annotations['PeptideSequence'].apply(lambda x : len(x))
    peptide_annotations = peptide_annotations.sort_values(['StartPosition','PeptideLength'], ascending=False)
    peptide_annotations = peptide_annotations[peptide_annotations['Ref_id'].notnull()]
    peptide_annotations = peptide_annotations.drop_duplicates('StartPosition')
    count = 0
    print(Counter(peptide_annotations['StartType']))
    for row in peptide_annotations.iterrows():
        print('*')
        count += 1
        print(count)
        print(row[1]['Ref_id'])
        print(row[1]['PeptideSequence'])
        #sequences.append(row[1]['PeptideSequence'])
        print(row[1]['AnnotationMessage'])
        print(row[1]['TSS']) 
        print(row[1]['AnnotationType'])
        print(row[1]['StartCodon'])
        print(row[1]['StartPosition'])


In [14]:
s5527_peptide_annotations = pd.read_csv(analysis_folder + '/annotations/H37Rv_S5527_peptide_annotations.csv')
s507_peptide_annotations = pd.read_csv(analysis_folder + '/annotations/H37Rv_S507_peptide_annotations.csv')
s5527_peptide_annotations[s5527_peptide_annotations['Strain_Nterm_Acetylated']==True]

,Unnamed: 0,ORF_id,Strain_Nterm_Acetylated,TSS,Enzymatic,StartCodon,StartPosition,PeptidePosition,StartType,AnnotationMessage,PeptideSequence,Ref_id,AnnotationType,VarId,OtherAnnotationType,OtherVarId
807,807,S5527_scaffold9_size227503|S5527_scaffold9_siz...,True,True,False,ATG,6.0,3,MAP-cleaved iMet,Non-tryptic N-terminus peptide ADVAESQENAPAER ...,ADVAESQENAPAER,NaN,Unspecified TSS identified,S5527_scaffold9_size227503|S5527_scaffold9_siz...,NaN,NaN
883,883,S5527_scaffold9_size227503|S5527_scaffold9_siz...,True,True,False,ATG,6.0,3,MAP-cleaved iMet,Non-tryptic N-terminus peptide ADVAESQENAPAER ...,ADVAESQENAPAER,I6X8D2,Annotated TSS validated,S5527_scaffold9_size227503|S5527_scaffold9_siz...,NaN,NaN
960,960,S5527_scaffold9_size227503|S5527_scaffold9_siz...,True,True,False,ATG,6.0,3,MAP-cleaved iMet,Non-tryptic N-terminus peptide ADVAESQENAPAER ...,ADVAESQENAPAER,NaN,Unspecified TSS identified,S5527_scaffold9_size227503|S5527_scaffold9_siz...,NaN,NaN
1036,1036,S5527_scaffold9_size227503|S5527_scaffold9_siz...,True,True,False,ATG,6.0,3,MAP-cleaved iMet,Non-tryptic N-terminus peptide ADVAESQENAPAER ...,ADVAESQENAPAER,I6X8D2,Annotated TSS validated,S5527_scaffold9_size227503|S5527_scaffold9_siz...,NaN,NaN
1112,1112,S5527_scaffold9_size227503|S5527_scaffold9_siz...,True,True,False,ATG,6.0,3,MAP-cleaved iMet,Non-tryptic N-terminus peptide ADVAESQENAPAER ...,ADVAESQENAPAER,NaN,Unspecified TSS identified,S5527_scaffold9_size227503|S5527_scaffold9_siz...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53144,53144,S5527_scaffold10_size173535|S5527_scaffold10_s...,True,True,False,ATG,9.0,4,MAP-cleaved iMet,Non-tryptic N-terminus peptide TSAPIPDITATPAWD...,TSAPIPDITATPAWDALRR,P9WN69,Annotated TSS validated,S5527_scaffold10_size173535|S5527_scaffold10_s...,NaN,NaN
53157,53157,S5527_scaffold10_size173535|S5527_scaffold10_s...,True,True,False,ATG,9.0,4,MAP-cleaved iMet,Non-tryptic N-terminus peptide TSAPIPDITATPAWD...,TSAPIPDITATPAWDALR,NaN,Unspecified TSS identified,S5527_scaffold10_size173535|S5527_scaffold10_s...,NaN,NaN
53161,53161,S5527_scaffold10_size173535|S5527_scaffold10_s...,True,True,False,ATG,9.0,4,MAP-cleaved iMet,Non-tryptic N-terminus peptide TSAPIPDITATPAWD...,TSAPIPDITATPAWDALRR,NaN,Unspecified TSS identified,S5527_scaffold10_size173535|S5527_scaffold10_s...,NaN,NaN
53174,53174,S5527_scaffold10_size173535|S5527_scaffold10_s...,True,True,False,ATG,9.0,4,MAP-cleaved iMet,Non-tryptic N-terminus peptide TSAPIPDITATPAWD...,TSAPIPDITATPAWDALR,P9WN69,Annotated TSS validated,S5527_scaffold10_size173535|S5527_scaffold10_s...,NaN,NaN


In [15]:
s5527_peptide_annotations_groups = pd.read_csv(analysis_folder + '/annotations/H37Rv_S5527_peptide_annotations_groups.csv')


In [16]:
#s5527_peptide_annotations_groups[s5527_peptide_annotations_groups['Identified S507_ML'] != s5527_peptide_annotations_groups['Identified S507_ST'] ]['PeptideSequence']

In [17]:
s5527_tss_counts= pd.read_csv(analysis_folder + 'stats/tss_counts_S5527.csv')

orf_id = s5527_tss_counts[s5527_tss_counts['ORF_start_count'] == 7]['ORF_id'].tolist()[0]
show_messages(s5527_peptide_annotations, orf_id)


Counter({'MAP-cleaved iMet': 6, 'Non-ATG iMet': 1})
*
1
P95029
SRWHAESSWAARVSLAHALIGWTR
Non-tryptic N-terminus peptide SRWHAESSWAARVSLAHALIGWTR following an GTG start-codon (initiator Methionine cleavage) downstream of annotated TSS site
True
Downstream TSS identified
GTG
6891.0
*
2
P95029
MLLWAVQRLIGGLSTIGAER
Non-ATG iMet peptide MLLWAVQRLIGGLSTIGAER identified downstream of annotated TSS site
True
Downstream TSS identified
GTG
6726.0
*
3
P95029
VRPGDEVDFR
Non-tryptic N-terminus peptide VRPGDEVDFR following an ATG start-codon (initiator Methionine cleavage) downstream of annotated TSS site
True
Downstream TSS identified
ATG
3858.0
*
4
P95029
TVTATAANATDTDMGR
Non-tryptic N-terminus peptide TVTATAANATDTDMGR following an TTG start-codon (initiator Methionine cleavage) downstream of annotated TSS site
True
Downstream TSS identified
TTG
3411.0
*
5
P95029
LPPVLSIR
Non-tryptic N-terminus peptide LPPVLSIR following an GTG start-codon (initiator Methionine cleavage) downstream of annotated TSS

In [18]:
s507_peptide_annotations[s507_peptide_annotations['AnnotationType'].notnull()]

,Unnamed: 0,ORF_id,Strain_Nterm_Acetylated,TSS,Enzymatic,StartCodon,StartPosition,PeptidePosition,StartType,AnnotationMessage,PeptideSequence,Ref_id,AnnotationType,VarId,OtherAnnotationType,OtherVarId
136,136,S507_scaffold1_size371796|S507_scaffold1_size3...,False,True,False,GTG,699.0,233,Non-enzymatic iMet,Non-enzymatic iMet peptide MLLAIDVR identified...,MLLAIDVR,NaN,Unspecified TSS identified,S507_scaffold1_size371796|S507_scaffold1_size3...,NaN,NaN
143,143,S507_scaffold1_size371796|S507_scaffold1_size3...,False,True,False,GTG,699.0,233,Non-enzymatic iMet,Non-enzymatic iMet peptide MLLAIDVR identified...,MLLAIDVR,P9WPA1,Annotated TSS validated,S507_scaffold1_size371796|S507_scaffold1_size3...,NaN,NaN
150,150,S507_scaffold1_size371796|S507_scaffold1_size3...,False,True,False,ATG,3.0,2,MAP-cleaved iMet,Non-tryptic N-terminus peptide PNTNPVAAWKALKEG...,PNTNPVAAWKALKEGNERFVAGR,NaN,Unspecified TSS identified,S507_scaffold1_size371796|S507_scaffold1_size3...,NaN,NaN
151,151,S507_scaffold1_size371796|S507_scaffold1_size3...,False,True,False,ATG,3.0,2,MAP-cleaved iMet,Non-tryptic N-terminus peptide PNTNPVAAWK foll...,PNTNPVAAWK,NaN,Unspecified TSS identified,S507_scaffold1_size371796|S507_scaffold1_size3...,NaN,NaN
158,158,S507_scaffold1_size371796|S507_scaffold1_size3...,False,True,False,ATG,3.0,2,MAP-cleaved iMet,Non-tryptic N-terminus peptide PNTNPVAAWKALKEG...,PNTNPVAAWKALKEGNERFVAGR,P9WPJ9,Annotated TSS validated,S507_scaffold1_size371796|S507_scaffold1_size3...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53057,53057,S507_scaffold9_size172227|S507_scaffold9_size1...,False,True,False,TTG,291.0,98,MAP-cleaved iMet,Non-tryptic N-terminus peptide LVCGHRAGRAAVTR ...,LVCGHRAGRAAVTR,P71568,Downstream TSS identified,S507_scaffold9_size172227|S507_scaffold9_size1...,NaN,NaN
53086,53086,S507_scaffold9_size172227|S507_scaffold9_size1...,False,True,False,ATG,63.0,21,Non-enzymatic iMet,Non-enzymatic iMet peptide MDITIVGK identified...,MDITIVGK,NaN,Unspecified TSS identified,S507_scaffold9_size172227|S507_scaffold9_size1...,NaN,NaN
53095,53095,S507_scaffold9_size172227|S507_scaffold9_size1...,False,True,False,ATG,63.0,21,Non-enzymatic iMet,Non-enzymatic iMet peptide MDITIVGK identified...,MDITIVGK,I6Y551,Annotated TSS validated,S507_scaffold9_size172227|S507_scaffold9_size1...,NaN,NaN
53116,53116,S507_scaffold9_size172227|S507_scaffold9_size1...,False,False,True,NaN,0.0,52,Enzymatic N-terminus non-TSS peptide,Enzymatic N-terminus non-TSS peptide PAVALGCCI...,PAVALGCCIPSAESDSCCVCSDGACMNILSRIFAR,P9WGK7,Upstream non-TSS peptide with no putative upst...,S507_scaffold9_size172227|S507_scaffold9_size1...,NaN,NaN


In [19]:
s507_tss_counts= pd.read_csv(analysis_folder + 'stats/tss_counts_S507.csv')
#print(s507_tss_counts.head())
orf_id = s507_tss_counts[s507_tss_counts['ORF_start_count'] == 6]['ORF_id'].tolist()[0]
print(orf_id)

#print(peptide_annotations['ORF_id'].tolist()[0])
#p = peptide_annotations[peptide_annotations['ORF_id']==ref_id]
show_messages(s507_peptide_annotations, orf_id)


S507_scaffold22_size60414|S507_scaffold22_size60414_recno_1131.0|(+)50646:53537
Counter({'Non-enzymatic iMet': 3, 'MAP-cleaved iMet': 2, 'Non-ATG iMet': 1})
*
1
O53166
MHSAAAHADGR
Non-enzymatic iMet peptide MHSAAAHADGR identified downstream of annotated TSS site
True
Downstream TSS identified
GTG
1374.0
*
2
O53166
VPSIAGPK
Non-tryptic N-terminus peptide VPSIAGPK following an GTG start-codon (initiator Methionine cleavage) downstream of annotated TSS site
True
Downstream TSS identified
GTG
1149.0
*
3
O53166
MSPEFGSTAAIFPIDEETIK
Non-enzymatic iMet peptide MSPEFGSTAAIFPIDEETIK identified downstream of annotated TSS site
True
Downstream TSS identified
ATG
957.0
*
4
O53166
VPPGTGIVHQVNIEYLASVVMTR
Non-tryptic N-terminus peptide VPPGTGIVHQVNIEYLASVVMTR following an GTG start-codon (initiator Methionine cleavage) downstream of annotated TSS site
True
Downstream TSS identified
GTG
567.0
*
5
O53166
MIADLFGRADAFERNVEIEYQR
Non-enzymatic iMet peptide MIADLFGRADAFERNVEIEYQR identified downstream of 

In [20]:
orfs = set(s5527_peptide_annotations[s5527_peptide_annotations['Ref_id'] == 'O53166']['ORF_id'].tolist())
assert len(orfs) == 1
orf_id = list(orfs)[0]
start_counts = s5527_tss_counts[s5527_tss_counts['ORF_id'] == orf_id]
show_messages(s5527_peptide_annotations, orf_id)
start_counts


Counter({'MAP-cleaved iMet': 1, 'Non-enzymatic iMet': 1, 'Non-ATG iMet': 1})
*
1
O53166
VPSIAGPK
Non-tryptic N-terminus peptide VPSIAGPK following an GTG start-codon (initiator Methionine cleavage) downstream of annotated TSS site
True
Downstream TSS identified
GTG
1149.0
*
2
O53166
MSPEFGSTAAIFPIDEETIK
Non-enzymatic iMet peptide MSPEFGSTAAIFPIDEETIK identified downstream of annotated TSS site
True
Downstream TSS identified
ATG
957.0
*
3
O53166
MVMQDFTGVPCIVDLATMR
Non-ATG iMet peptide MVMQDFTGVPCIVDLATMR identified downstream of annotated TSS site
True
Downstream TSS identified
GTG
309.0


,Unnamed: 0,ORF_id,ORF_start_count
16,21013,S5527_scaffold3_size359573|S5527_scaffold3_siz...,3


In [21]:
orfs = set(s507_peptide_annotations[s507_peptide_annotations['Ref_id'] == 'P95029']['ORF_id'].tolist())
assert len(orfs) == 1
orf_id = list(orfs)[0]
start_counts = s507_tss_counts[s507_tss_counts['ORF_id'] == orf_id]
show_messages(s507_peptide_annotations, orf_id)
start_counts

Counter({'MAP-cleaved iMet': 4, 'Non-ATG iMet': 1})
*
1
P95029
MLLWAVQRLIGGLSTIGAER
Non-ATG iMet peptide MLLWAVQRLIGGLSTIGAER identified downstream of annotated TSS site
True
Downstream TSS identified
GTG
6726.0
*
2
P95029
VRPGDEVDFR
Non-tryptic N-terminus peptide VRPGDEVDFR following an ATG start-codon (initiator Methionine cleavage) downstream of annotated TSS site
True
Downstream TSS identified
ATG
3858.0
*
3
P95029
AGPLAVVLDAPDVR
Non-tryptic N-terminus peptide AGPLAVVLDAPDVR following an GTG start-codon (initiator Methionine cleavage) downstream of annotated TSS site
True
Downstream TSS identified
GTG
2787.0
*
4
P95029
LPPVLSIR
Non-tryptic N-terminus peptide LPPVLSIR following an GTG start-codon (initiator Methionine cleavage) downstream of annotated TSS site
True
Downstream TSS identified
GTG
666.0
*
5
P95029
IGAAGTLVAR
Non-tryptic N-terminus peptide IGAAGTLVAR following an TTG start-codon (initiator Methionine cleavage) downstream of annotated TSS site
True
Downstream TSS identif

,Unnamed: 0,ORF_id,ORF_start_count
1,9638,S507_scaffold13_size114854|S507_scaffold13_siz...,5


In [22]:
# create supplementary table of tss counts
s507_tss_counts['Strain' ] = 'S507'
s5527_tss_counts['Strain' ] = 'S5527'

combined = pd.concat([s507_tss_counts, s5527_tss_counts])
del combined['Unnamed: 0']
combined.to_excel('supplementary/s4.xlsx')
combined.head()

,ORF_id,ORF_start_count,Strain
0,S507_scaffold22_size60414|S507_scaffold22_size...,6,S507
1,S507_scaffold13_size114854|S507_scaffold13_siz...,5,S507
2,S507_scaffold7_size206714|S507_scaffold7_size2...,4,S507
3,S507_scaffold11_size165255|S507_scaffold11_siz...,4,S507
4,S507_scaffold12_size150664|S507_scaffold12_siz...,4,S507


In [23]:
exclusive_tss = pd.read_csv(analysis_folder + 'annotations/H37Rv_exclusive_tss.csv')
tss_diff = pd.read_csv(analysis_folder +'/annotations/export/H37Rv/Combined_Proteins_TSS_Differences.csv')
del tss_diff['Unnamed: 0']
exclusive_tss= pd.merge(exclusive_tss, tss_diff, left_on='BLASTP', right_on='BLASTP')
exclusive_tss = exclusive_tss.sort_values(['BLASTP'])
exclusive_tss.to_excel('supplementary/s5.xlsx')
shutil.copyfile(analysis_folder + 'annotations/H37Rv_exclusive_tss.fasta', "supplementary/s6.fasta")

'supplementary/s6.fasta'

## 3.2.6-7 Differences in translation start sites identified between strains. 

In [24]:
shutil.copyfile(analysis_folder + 'annotations/export/H37Rv/pathview/mtu01100.tss_differences.png', 'figures/figure3.png')

print(len(exclusive_tss))
print(Counter(exclusive_tss['Comparison']))
_5527 = exclusive_tss[exclusive_tss['Comparison'] == "S5527 vs S507"]['ORF_ids'].tolist()
_507 = exclusive_tss[exclusive_tss['Comparison'] == "S507 vs S5527"]['ORF_ids'].tolist()

intersect = set(_5527) & set(_507)

exclusive_tss_both = exclusive_tss[exclusive_tss['ORF_ids'].isin(intersect)]

print('TSS variant sequences : ',len(exclusive_tss_both))
exclusive_tss_both.to_excel('supplementary/s7.xlsx')
print(set(exclusive_tss_both['Ref_Ids'].tolist()))
targets = exclusive_tss_both['BLASTP'].unique()

s507_proteoforms = exclusive_tss_both[exclusive_tss_both['Comparison'] == 'S507 vs S5527']
s5527_proteoforms = exclusive_tss_both[exclusive_tss_both['Comparison'] == 'S5527 vs S507']

s507 = s507_proteoforms.groupby('BLASTP')['VarId'].count()
s507.index.name='S507'
s5527 = s5527_proteoforms.groupby('BLASTP')['VarId'].count()
s5527.index.name='S5527'
combined = pd.concat([s5527,s507], axis=1)
combined.columns = ['S5527','S507']
combined = combined.reset_index()

combined = pd.merge(combined, tss_diff, left_on='index', right_on='BLASTP')
#del combined['Unnamed: 0']
del combined['Comparisons']
del combined['index']
combined = combined.set_index('BLASTP')
combined = combined.reset_index()
combined.rename(columns={'BLASTP':"Protein"}, inplace=True)

#combined = combined[combined.columns[::-1]]
#tss_diff.to_excel('supplementary/s8.xlsx')
combined.to_excel('tables/table3.xlsx')

combined # number of exclusive proteoforms in each strain

235
Counter({'S5527 vs S507': 132, 'S507 vs S5527': 103})
TSS variant sequences :  17
{'P9WQD7', 'O53580', 'P9WKC7', 'P9WI57', 'P9WG47', 'I6Y231', 'P95029'}


,Protein,S5527,S507,Gene Name,Protein Name
0,I6Y231,1,1,mas,Probable multifunctional mycocerosic acid synt...
1,O53580,1,1,fadD32,Long-chain-fatty-acid--AMP ligase FadD32
2,P95029,3,1,fas,Probable fatty acid synthase Fas (Fatty acid s...
3,P9WG47,2,1,gyrA,DNA gyrase subunit A
4,P9WI57,1,1,pnp,Polyribonucleotide nucleotidyltransferase
5,P9WKC7,1,1,tgs2,Probable diacyglycerol O-acyltransferase tgs2
6,P9WQD7,1,1,kasB,3-oxoacyl-[acyl-carrier-protein] synthase 2


In [25]:
domains = pd.read_csv(analysis_folder + 'fasta/proteins.fasta.tsv',sep='\t')

domains = domains.sort_values('6')

In [26]:
domains = domains[domains['11'] != '-']

In [27]:
#NB note - heat shock protein domains, missing in hypo: S5527_scaffold5_size225283_recno_5478.0. Important proteoform differences


In [28]:

def summarize(reference):

    print("\n****Hypovirulent************************************************************************\n")
    _ = s507_peptide_annotations[( s507_peptide_annotations['Ref_id']==reference ) & ( s507_peptide_annotations['TSS']==True )].drop_duplicates('PeptideSequence')
    records = _[['ORF_id', 'PeptideSequence','StartPosition','StartCodon','AnnotationType', 'AnnotationMessage','PeptidePosition']].to_dict(orient='records')
    s507_unseen = set()
    s507_seen = set()
    s507_exclusive= set()
    s507_exclusive_count = 0
    s507_shared_count = 0
    for rec in records:
        
       
        print("\nORF record: ")
        print(rec)
        orf_id = rec['ORF_id']
        #print(domains[(domains['0'] == orf_id)] )
        
        aa_start = (rec['StartPosition']  )/3
        print("Translation start: ", aa_start)
        s507_domains_= domains[(domains['0'] == orf_id) & (domains['6'] < aa_start)  ]
        
        s507_domains_seen = domains[(domains['0'] == orf_id) & (domains['6'] >= aa_start)  ]['11'].tolist()
        print("Affected domains: ", s507_domains_['11'].tolist())
        s507_seen.update(s507_domains_seen) 
        
        if rec['PeptideSequence'] in exclusive_tss['PeptideSequence'].tolist():
            s507_exclusive.update(s507_domains_seen)
            print("ORF translatioon is exclusive")
            s507_exclusive_count +=1
        else:
            print("ORF translation is shared")
            s507_shared_count +=1
        
        all_s507_domains_= domains[(domains['0'] == orf_id)  ]
        s507_unseen.update(s507_domains_['6'].tolist())
    try:
        print(all_s507_domains_[['11','3', '4','5', '6','7']])
    except:
        pass
    print("\n****Hypervirulent************************************************************************\n")
    _ = s5527_peptide_annotations[( s5527_peptide_annotations['Ref_id']==reference ) & ( s5527_peptide_annotations['TSS']==True )].drop_duplicates('PeptideSequence')
    records = _[['ORF_id','PeptideSequence','StartPosition','StartCodon','AnnotationType', 'AnnotationMessage','PeptidePosition']].to_dict(orient='records')
    s5527_unseen = set()
    s5527_seen = set()
    s5527_exclusive = set()
    s5527_exclusive_count = 0
    s5527_shared_count = 0
    for rec in records:
        

        print("\nORF record: ")
        print(rec)
        orf_id = rec['ORF_id']
        aa_start = (rec['StartPosition']  )/3
        print("Tranlation start: ", aa_start)
        s5527_domains_= domains[(domains['0'] == orf_id) & (domains['6'] < aa_start)   ]
        s5527_domains_seen = domains[(domains['0'] == orf_id) & (domains['6'] >= aa_start)  ]['11'].tolist()
        print("Affected domains: ", s5527_domains_['11'].tolist())

        s5527_seen.update(s5527_domains_seen)
        if rec['PeptideSequence'] in exclusive_tss['PeptideSequence'].tolist():
            s5527_exclusive.update(s5527_domains_seen)
            print("ORF translation is exclusve")
            s5527_exclusive_count +=1
        else:
            print("ORF translation is shared")
            s5527_shared_count += 1
        all_s5527_domains_= domains[(domains['0'] == orf_id)  ]
        s5527_unseen.update(s5527_domains_['6'])
    try:
        print(all_s5527_domains_[['11','3','4', '5', '6','7']])
    except:
        pass
    
    print("\nS507 specific domains not affected in any ORF: ", s507_seen-s5527_seen)
    print("S507 specific domains not affected in any exclusive ORF: ", s507_exclusive-s5527_exclusive)
    print("Number of translated sequences exclusively identified in S507: ", s507_exclusive_count)
    print("Number of translated sequences also identified in S5527: ", s507_shared_count)
    
    print("\nS5527 specific domains not affected in any ORF: ", s5527_seen-s507_seen)
    print("S5527 specific domains not affected in any  exclusive ORF: ", s5527_exclusive-s507_exclusive)
    print("Number of translated sequences exclusively identified in S5527: ", s5527_exclusive_count)
    print("Number of translated sequences also identified in S507: ", s5527_shared_count)
#summarize('P9WI57')
#summarize('P95029')
#summarize('P9WG47')
#summarize('P9WKC7')
#summarize('I6Y231')

#summarize('P9WQD7')
#summarize('O53580')


# 3.2.8 Protein XseB exclusively identified in the viruklent strain (Exclusive protein identifications)

In [29]:
sequences = SeqIO.to_dict(SeqIO.parse(analysis_folder + '/strains/all_mapped_trans_orfs.fasta', format = 'fasta'))

In [30]:
s507_peptides = pd.read_csv(analysis_folder + '/annotations/H37Rv_peptides_S507_vs_S5527.csv')
recs = s507_peptides[s507_peptides['SpecificPeptidesExclusive'] == True].to_dict(orient='records')
pprint.pprint(recs)
print(sequences[recs[0]['ORF_id']].format('fasta'))
# a signle peptide identification, and no signification results using NCBI BLAST - excluded

[{'Acetylated S507_ML': False,
  'Acetylated S507_MLexp': False,
  'Acetylated S507_ST': False,
  'Acetylated S5527_ML': False,
  'Acetylated S5527_MLexp': False,
  'Acetylated S5527_ST': False,
  'BLASTP': nan,
  'Identified S507_ML': False,
  'Identified S507_MLexp': False,
  'Identified S507_ST': True,
  'Identified S5527_ML': False,
  'Identified S5527_MLexp': False,
  'Identified S5527_ST': False,
  'ORF_id': 'S507_scaffold1_size371796|S507_scaffold1_size371796_recno_9181.0|(-)14899:15330',
  'ORF_ids': 'S507_scaffold1_size371796|S507_scaffold1_size371796_recno_9181.0|(-)14899:15330;S5527_scaffold9_size227503|S5527_scaffold9_size227503_recno_3627.0|(+)117207:117638',
  'OtherStrainIdentified': False,
  'OtherStrainPredicted': True,
  'PeptideSequence': 'GLLTRRGQAGIMQRR',
  'Ref_Ids': nan,
  'SpecificPeptidesExclusive': True,
  'TSS': False,
  'VarId': nan,
  'VarLength': nan}]
>S507_scaffold1_size371796|S507_scaffold1_size371796_recno_9181.0|(-)14899:15330 <unknown description>
AT

In [31]:
# Protein XseB

s5527_peptides = pd.read_csv(analysis_folder + '/annotations/H37Rv_peptides_S5527_vs_S507.csv')
recs = s5527_peptides[s5527_peptides['SpecificPeptidesExclusive'] == True].to_dict(orient='records')
pprint.pprint(recs)
print(sequences[recs[0]['ORF_id']].format('fasta'))

[{'Acetylated S507_ML': False,
  'Acetylated S507_MLexp': False,
  'Acetylated S507_ST': False,
  'Acetylated S5527_ML': False,
  'Acetylated S5527_MLexp': False,
  'Acetylated S5527_ST': False,
  'BLASTP': 'P9WF29',
  'Identified S507_ML': False,
  'Identified S507_MLexp': False,
  'Identified S507_ST': False,
  'Identified S5527_ML': True,
  'Identified S5527_MLexp': False,
  'Identified S5527_ST': False,
  'ORF_id': 'S5527_scaffold18_size93797|S5527_scaffold18_size93797_recno_728.0|(+)29465:29746',
  'ORF_ids': 'S507_scaffold27_size47148|S507_scaffold27_size47148_recno_453.0|(+)31379:31660;S5527_scaffold18_size93797|S5527_scaffold18_size93797_recno_728.0|(+)29465:29746',
  'OtherStrainIdentified': False,
  'OtherStrainPredicted': True,
  'PeptideSequence': 'DELMEVVR',
  'Ref_Ids': 'P9WF29',
  'SpecificPeptidesExclusive': True,
  'TSS': False,
  'VarId': nan,
  'VarLength': nan},
 {'Acetylated S507_ML': False,
  'Acetylated S507_MLexp': False,
  'Acetylated S507_ST': False,
  'Acetyl

# 3.2.9 Coding polymorphisms identified in the hypo virulent strain

In [32]:
s5527_variants = s5527_peptides[s5527_peptides['OtherStrainPredicted']==False]
print("Numver of variant peptides in s507: ", len(s5527_variants))
s507_variants = s507_peptides[s507_peptides['OtherStrainPredicted']==False]
print("Numver of variant peptides in s507: ", len(s507_variants))
variant_peptides = s507_variants['PeptideSequence'].tolist() + s5527_variants['PeptideSequence'].tolist()


Numver of variant peptides in s507:  0
Numver of variant peptides in s507:  2


In [33]:
H37Rv_results = pd.read_csv(analysis_folder + '/H37Rv_combined.csv',sep='\t')
CDC1551_results = pd.read_csv(analysis_folder + '/CDC1551_combined.csv',sep='\t')
pos_col='All peptides group '
pos=False
_cols = ['id']
for col in H37Rv_results.columns.tolist():
    if col.startswith(pos_col):
        pos=True
    elif pos == True:
        _cols.append(col)


print(_cols)

#print(H37Rv_results.columns.tolist())
combined = H37Rv_results[_cols]
combined

['id', 'Row paralogous specific peptides', 'Row non-paralogous specific peptides', 'All peptides strain S5527', 'Specific peptides strain S5527', 'Exclusive peptides strain S5527', 'Novel specific peptides strain S5527', 'Annotated specific peptides strain S5527', 'Reference BLAST strain S5527', 'Non-genomic peptides strain S5527', 'Translated orfs strain S5527', 'Variant orfs strain S5527', 'Annotation type strain S5527', 'Best orf-reference blast evalue strain S5527', 'Best orf-reference blast match S5527', 'All peptides strain S507', 'Specific peptides strain S507', 'Exclusive peptides strain S507', 'Novel specific peptides strain S507', 'Annotated specific peptides strain S507', 'Reference BLAST strain S507', 'Non-genomic peptides strain S507', 'Translated orfs strain S507', 'Variant orfs strain S507', 'Annotation type strain S507', 'Best orf-reference blast evalue strain S507', 'Best orf-reference blast match S507', 'Variant peptide BLAST strain S5527', 'Variant peptide feature ov

,id,Row paralogous specific peptides,Row non-paralogous specific peptides,All peptides strain S5527,Specific peptides strain S5527,Exclusive peptides strain S5527,Novel specific peptides strain S5527,Annotated specific peptides strain S5527,Reference BLAST strain S5527,Non-genomic peptides strain S5527,...,UniProtKB-ID,Reference ICDS evidence,Strain ICDS intrastrain evidence S5527,Strain ICDS interstrain exclusive evidence S5527,Strain ICDS intrastrain peptide validation S5527,Strain ICDS interstrain exclusive peptide validation S5527,Strain ICDS intrastrain evidence S507,Strain ICDS interstrain exclusive evidence S507,Strain ICDS intrastrain peptide validation S507,Strain ICDS interstrain exclusive peptide validation S507
0,36,NaN,ALVTGAAGFIGSTLVDR\nDYVFVDDVVDAFVR\nIVHTSSGGSIY...,ALVTGAAGFIGSTLVDR\nDYVFVDDVVDAFVR\nIVHTSSGGSIY...,ALVTGAAGFIGSTLVDR\nDYVFVDDVVDAFVR\nIVHTSSGGSIY...,NaN,NaN,ALVTGAAGFIGSTLVDR\nDYVFVDDVVDAFVR\nIVHTSSGGSIY...,P9WN67,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,37,NaN,ALAAPVSAVPPSYVQAR\nFDETAASVLFPDNPVAR\nHSIADPAD...,FDETAASVLFPDNPVAR\nHSIADPADIEFVPTTHGEMTSADLR\n...,FDETAASVLFPDNPVAR\nHSIADPADIEFVPTTHGEMTSADLR\n...,NaN,NaN,FDETAASVLFPDNPVAR\nHSIADPADIEFVPTTHGEMTSADLR\n...,P9WIK9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38,NaN,DGPVLLAHTLDDQAETVLLGLGR\nELGLTAWQDPHNTDR\nGQEL...,DGPVLLAHTLDDQAETVLLGLGR\nELGLTAWQDPHNTDR\nGQEL...,DGPVLLAHTLDDQAETVLLGLGR\nELGLTAWQDPHNTDR\nGQEL...,NaN,NaN,DGPVLLAHTLDDQAETVLLGLGR\nELGLTAWQDPHNTDR\nGQEL...,P9WG53,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39,NaN,AQSAQYAFILDR\nAQSAQYAFILDRMSQQVDADEHRVALLR\nKT...,AQSAQYAFILDR\nAQSAQYAFILDRMSQQVDADEHRVALLR\nKT...,AQSAQYAFILDR\nAQSAQYAFILDRMSQQVDADEHRVALLR\nKT...,NaN,MSVSTLMDGRIDHVELSARVAWMSESQLASEILVIADLAR,AQSAQYAFILDR\nAQSAQYAFILDRMSQQVDADEHRVALLR\nKT...,P9WJD5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40,NaN,AQSAQYAFILDR\nAQSAQYAFILDRMSQQVDADEHRVALLR\nKT...,AQSAQYAFILDR\nAQSAQYAFILDRMSQQVDADEHRVALLR\nKT...,AQSAQYAFILDR\nAQSAQYAFILDRMSQQVDADEHRVALLR\nKT...,NaN,MSVSTLMDGRIDHVELSARVAWMSESQLASEILVIADLAR,AQSAQYAFILDR\nAQSAQYAFILDRMSQQVDADEHRVALLR\nKT...,P9WJD5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2322,2358,NaN,ADYGPLVTGAALAR\nAGPALACGNAFVLKPSER\nAVGFVGSSDI...,ADYGPLVTGAALAR\nAGPALACGNAFVLKPSER\nAVGFVGSSDI...,ADYGPLVTGAALAR\nAGPALACGNAFVLKPSER\nAVGFVGSSDI...,NaN,NHMIVMPDADLGQAVDALIGAGYGSAGER,ADYGPLVTGAALAR\nAGPALACGNAFVLKPSER\nAVGFVGSSDI...,O53816,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2323,2359,NaN,AGSAEPTADLAMWPPAGAVPVEVADGYQQLAER\nAHAVAVPPMFS...,AGIDPTGLR\nAGSAEPTADLAMWPPAGAVPVEVADGYQQLAER\n...,AGSAEPTADLAMWPPAGAVPVEVADGYQQLAER\nAHAVAVPPMFS...,NaN,TGGFVDGVGDFDPAFFGVGPSEALAMDPQQR,AGSAEPTADLAMWPPAGAVPVEVADGYQQLAER\nAHAVAVPPMFS...,I6XD69,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2324,2360,NaN,AAYTNAGMQPSEVDYVEAHGTGTLLGDPIEAR\nAGVSSFGFGGTN...,AAYTNAGMQPSEVDYVEAHGTGTLLGDPIEAR\nALAAGQHAPGVV...,AAYTNAGMQPSEVDYVEAHGTGTLLGDPIEAR\nALAAGQHAPGVV...,NaN,ALAAGQHAPGVVNPAEGSPGPGTVFVYSGR\nQTVIAGPTEQIDELITR,AAYTNAGMQPSEVDYVEAHGTGTLLGDPIEAR\nDADKPAALWILT...,P9WQE7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2325,2361,LADILTR\nYAADTRDASVRCAR,AMLAPESLTADER\nLADILTR\nLTVVAAAEAR\nSLSHVGLTSD...,LTVVAAAEAR\nVLAENVDADEVTAAAR\nYAADTRDASVRCAR,LTVVAAAEAR\nVLAENVDADEVTAAAR\nYAADTRDASVRCAR,NaN,NaN,LTVVAAAEAR\nVLAENVDADEVTAAAR\nYAADTRDASVRCAR,O33269,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
variants = combined[combined['Specific peptides - all strains'].apply(lambda x : len(list(set(str(x).split('\n')) & set( variant_peptides))) > 0)]
variants = variants.dropna(how='all', axis=1)
#variant_peptides
variants.to_excel('supplementary/s8.xlsx')
variants

,id,Row non-paralogous specific peptides,All peptides strain S5527,Specific peptides strain S5527,Novel specific peptides strain S5527,Annotated specific peptides strain S5527,Reference BLAST strain S5527,Translated orfs strain S5527,Variant orfs strain S5527,Annotation type strain S5527,...,Specific novel peptides - all strains,ORF ids - all strains,Reference proteins mapped - all strains,Reference proteins mapped count - all strains,Reference entries mapped - all strains,Variant peptides reference blast - all strains,Variant peptides reference features - all strains,Non-tryptic nterm peptides - all strains,Reference BLAST - all strains,Identifier
898,934,ADGAAIDSLLGPEMK\nAEGLLAVTGDGAHAAR\nAEGLQVSLVNS...,ADGAAIDSLLGPEMK\nAEGLLAVTGDGAHAAR\nAEGLQVSLVNS...,ADGAAIDSLLGPEMK\nAEGLLAVTGDGAHAAR\nAEGLQVSLVNS...,RAGVFGAIPPGSR,ADGAAIDSLLGPEMK\nAEGLLAVTGDGAHAAR\nAEGLQVSLVNS...,P9WPK3,>S5527_scaffold19_size115875|S5527_scaffold19_...,>S5527_scaffold19_size115875|S5527_scaffold19_...,Upstream non-TSS peptide with no putative upst...,...,RAGVFGAIPPGSR\nYGTATTFSQAR,S5527_scaffold19_size115875|S5527_scaffold19_s...,>sp|P9WPK3|CARB_MYCTU Carbamoyl phosphate synt...,1.0,P9WPK3,***Results for peptide YGTATTFSQAR in record s...,***Results for peptide YGTATTFSQAR in record s...,RAGVFGAIPPGSR,P9WPK3,(protein group 934)
1492,1528,AEIDHWQNVDPALFGR\nFVYEDDDVVAFLTIEPMTQGHTLVVPR\...,AEIDHWQNVDPALFGR\nFVYEDDDVVAFLTIEPMTQGHTLVVPR\...,AEIDHWQNVDPALFGR\nFVYEDDDVVAFLTIEPMTQGHTLVVPR\...,NaN,AEIDHWQNVDPALFGR\nFVYEDDDVVAFLTIEPMTQGHTLVVPR\...,P9WML3,>S5527_scaffold27_size39624|S5527_scaffold27_s...,NaN,NaN,...,PGGAVATSYAQRVSIFTK,S5527_scaffold27_size39624|S5527_scaffold27_si...,>sp|P9WML3|YHI1_MYCTU Uncharacterized HIT-like...,1.0,P9WML3,NaN,NaN,NaN,P9WML3,(protein group 1528)


# 3.2.10 ICDSs detected in the hypovirulent strain.

In [35]:
# Identify columns that contain "ICDS" in their name
fs_cols = [col for col in H37Rv_results.columns if "ICDS" in col]

# Include the "BLASTP" column in the list of columns to select
selected_cols = fs_cols + ["Reference entries mapped - all strains", "ORF ids - all strains"]

# Select the desired columns from the DataFrames
fs = H37Rv_results[selected_cols]
fs2 = CDC1551_results[selected_cols]

# Function to patch the 'Reference ICDS evidence' column
# patch for a known bug causing ICDS for another strain to be reported if the current strain did not yield a reference ICDS
def clean_reference_icds(series, exclude_str):
    """
    Cleans the 'Reference ICDS evidence' column by removing entries containing the exclude string.
    """
    return series.apply(lambda x: None if exclude_str in str(x) else x)

# Apply the patch to the 'Reference ICDS evidence' column
fs['Reference ICDS evidence'] = clean_reference_icds(fs['Reference ICDS evidence'], exclude_str="_MYCTO")
fs2['Reference ICDS evidence'] = clean_reference_icds(fs2['Reference ICDS evidence'], exclude_str="_MYCTU")

# Function to preprocess the DataFrame
def preprocess_fs(df, icds_cols):
    """
    Preprocesses the DataFrame by:
    - Dropping rows where all ICDS columns are NaN
    - Removing duplicate rows
    - Resetting the index
    """
    df = df.dropna(subset=icds_cols, how='all')  # Drop rows where all "ICDS" columns are NaN
    df = df.drop_duplicates()                   # Remove duplicate rows
    df = df.reset_index(drop=True)              # Reset the index
    return df

# Preprocess the DataFrames
fs = preprocess_fs(fs, fs_cols)
fs2 = preprocess_fs(fs2, fs_cols)


<ipython-input-35-85dd550f0945>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fs['Reference ICDS evidence'] = clean_reference_icds(fs['Reference ICDS evidence'], exclude_str="_MYCTO")
<ipython-input-35-85dd550f0945>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fs2['Reference ICDS evidence'] = clean_reference_icds(fs2['Reference ICDS evidence'], exclude_str="_MYCTU")


In [36]:
fs.head()

,ICDS,Reference ICDS evidence,Strain ICDS intrastrain evidence S5527,Strain ICDS interstrain exclusive evidence S5527,Strain ICDS intrastrain peptide validation S5527,Strain ICDS interstrain exclusive peptide validation S5527,Strain ICDS intrastrain evidence S507,Strain ICDS interstrain exclusive evidence S507,Strain ICDS intrastrain peptide validation S507,Strain ICDS interstrain exclusive peptide validation S507,Reference entries mapped - all strains,ORF ids - all strains
0,NaN,tr|P96248|P96248_MYCTU and tr|O07794|O07794_MY...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O07794;P96248,S5527_scaffold9_size227503|S5527_scaffold9_siz...
1,NaN,sp|P9WKC5|TGS3_MYCTU and tr|O05878|O05878_MYCT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P9WKC5;O05878,S5527_scaffold8_size164766|S5527_scaffold8_siz...
2,S5527,NaN,S5527_scaffold8_size164766|S5527_scaffold8_siz...,S5527_scaffold8_size164766|S5527_scaffold8_siz...,+,-,NaN,NaN,NaN,NaN,P96901,S5527_scaffold8_size164766|S5527_scaffold8_siz...
3,NaN,sp|O07718|ACEAA_MYCTU and sp|O07717|ACEAB_MYCT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O07717;O07718,S5527_scaffold7_size169925|S5527_scaffold7_siz...
4,NaN,tr|P95120|P95120_MYCTU and tr|I6Y259|I6Y259_MY...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I6Y259;P95120,S5527_scaffold4_size253745|S5527_scaffold4_siz...


In [37]:
reference_icds = fs[fs['Reference ICDS evidence'].notnull()]
reference_icds = reference_icds[reference_icds['Reference ICDS evidence'].apply (lambda x : "_MYCTU" in x)] 
reference_icds = reference_icds.reset_index()
del reference_icds['index']

reference_icds2 = fs2[fs2['Reference ICDS evidence'].notnull()]
reference_icds2 = reference_icds2[reference_icds2['Reference ICDS evidence'].apply (lambda x : "_MYCTO" in x)] # patch for a known bug causing ICDS for another strain to be reported if the current strain did not yield a reference ICDS
reference_icds2 = reference_icds2.reset_index()
del reference_icds2['index']

interstrain_icds =  fs[(fs['Strain ICDS intrastrain evidence S5527'].notnull()) | (fs['Strain ICDS intrastrain evidence S507'].notnull()) ] 
interstrain_icds = interstrain_icds.reset_index()
del interstrain_icds['index']

interstrain_icds.to_excel('supplementary/s9.xlsx')
reference_icds.to_excel('supplementary/s10.xlsx') 
reference_icds2.to_excel('supplementary/s13.xlsx')


In [38]:
# ICDS in hypovirulent strain relative to H37Rv
interstrain_icds[interstrain_icds['Strain ICDS intrastrain evidence S507'].notnull()]


,ICDS,Reference ICDS evidence,Strain ICDS intrastrain evidence S5527,Strain ICDS interstrain exclusive evidence S5527,Strain ICDS intrastrain peptide validation S5527,Strain ICDS interstrain exclusive peptide validation S5527,Strain ICDS intrastrain evidence S507,Strain ICDS interstrain exclusive evidence S507,Strain ICDS intrastrain peptide validation S507,Strain ICDS interstrain exclusive peptide validation S507,Reference entries mapped - all strains,ORF ids - all strains
2,S507,sp|Q79FP0|PG29_MYCTU and tr|O53158|O53158_MYCT...,NaN,NaN,NaN,NaN,S507_scaffold19_size115010|S507_scaffold19_siz...,S507_scaffold19_size115010|S507_scaffold19_siz...,-,-,O53158;Q79FP0,S5527_scaffold19_size115875|S5527_scaffold19_s...
3,S507,NaN,NaN,NaN,NaN,NaN,S507_scaffold1_size371796|S507_scaffold1_size3...,S507_scaffold1_size371796|S507_scaffold1_size3...,-,-,P9WQ51;I6YGC8,S5527_scaffold29_size34899|S5527_scaffold29_si...
4,S507,NaN,NaN,NaN,NaN,NaN,S507_scaffold14_size153262|S507_scaffold14_siz...,S507_scaffold14_size153262|S507_scaffold14_siz...,+,-,O07738,S5527_scaffold7_size169925|S5527_scaffold7_siz...


In [39]:
interstrain_icds[interstrain_icds['Strain ICDS interstrain exclusive evidence S507'].notnull()].values[0] #Novel gene fusion in viruelent strain

array(['S507',
       'sp|Q79FP0|PG29_MYCTU and tr|O53158|O53158_MYCTU did not align to each other, but both aligned to S5527_scaffold19_size115875|S5527_scaffold19_size115875_recno_2796.0|(-)111534:114620 with an evalue cutoff or 0.0001',
       nan, nan, nan, nan,
       'S507_scaffold19_size115010|S507_scaffold19_size115010_recno_1987.0|(-)112611:113762 and S507_scaffold19_size115010|S507_scaffold19_size115010_recno_2624.0|(-)110675:112714 did not align to each other, but both aligned to S5527_scaffold19_size115875|S5527_scaffold19_size115875_recno_2796.0|(-)111534:114620 with an evalue cutoff or 0.0001',
       'S507_scaffold19_size115010|S507_scaffold19_size115010_recno_1987.0|(-)112611:113762 and S507_scaffold19_size115010|S507_scaffold19_size115010_recno_2624.0|(-)110675:112714 did not align to each other, but both aligned to S5527_scaffold19_size115875|S5527_scaffold19_size115875_recno_2796.0|(-)111534:114620 with an evalue cutoff or 0.0001',
       '-', '-', 'O53158;Q79FP0',
 

In [40]:
interstrain_icds[interstrain_icds['Strain ICDS interstrain exclusive evidence S507'].notnull()].values[1] 

array(['S507', nan, nan, nan, nan, nan,
       'S507_scaffold1_size371796|S507_scaffold1_size371796_recno_7040.0|(+)366807:368621 and S507_scaffold1_size371796|S507_scaffold1_size371796_recno_2394.0|(+)368413:368706 did not align to each other, but both aligned to S5527_scaffold29_size34899|S5527_scaffold29_size34899_recno_540.0|(+)28791:30557;S5527_scaffold29_size34899|S5527_scaffold29_size34899_recno_344.0|(+)23501:25399 with an evalue cutoff or 0.0001',
       'S507_scaffold1_size371796|S507_scaffold1_size371796_recno_7040.0|(+)366807:368621 and S507_scaffold1_size371796|S507_scaffold1_size371796_recno_2394.0|(+)368413:368706 did not align to each other, but both aligned to S5527_scaffold29_size34899|S5527_scaffold29_size34899_recno_540.0|(+)28791:30557;S5527_scaffold29_size34899|S5527_scaffold29_size34899_recno_344.0|(+)23501:25399 with an evalue cutoff or 0.0001',
       '-', '-', 'P9WQ51;I6YGC8',
       'S5527_scaffold29_size34899|S5527_scaffold29_size34899_recno_344.0|(+)23501:2

In [41]:
# peptide spanning the ICDS identified in the same strain - frameshift of sequencing error
interstrain_icds[interstrain_icds['Strain ICDS interstrain exclusive evidence S507'].notnull()].values[2] 

array(['S507', nan, nan, nan, nan, nan,
       'S507_scaffold14_size153262|S507_scaffold14_size153262_recno_3256.0|(-)110727:111839 and S507_scaffold14_size153262|S507_scaffold14_size153262_recno_4208.0|(-)110444:111145 did not align to each other, but both aligned to S5527_scaffold7_size169925|S5527_scaffold7_size169925_recno_2325.0|(+)25638:27032 with an evalue cutoff or 0.0001\n***Frameshift validation peptide MTMVANALGTPPADMIK***\nMTMVANALGTPP matched at aa postion 231-242 in S507_scaffold14_size153262|S507_scaffold14_size153262_recno_3256.0|(-)110727:111839 (genomic coordinates: (-)111114:111149 in contig S507_scaffold14_size153262)\nDMIK matched at aa postion 10-16 in S507_scaffold14_size153262|S507_scaffold14_size153262_recno_4208.0|(-)110444:111145 (genomic coordinates: (-)111098:111118 in contig S507_scaffold14_size153262)',
       'S507_scaffold14_size153262|S507_scaffold14_size153262_recno_3256.0|(-)110727:111839 and S507_scaffold14_size153262|S507_scaffold14_size153262_recn

# 3.2.11 ICDSs detected in the virulent strain.

In [42]:
interstrain_icds[interstrain_icds['Strain ICDS interstrain exclusive evidence S5527'].notnull()]

,ICDS,Reference ICDS evidence,Strain ICDS intrastrain evidence S5527,Strain ICDS interstrain exclusive evidence S5527,Strain ICDS intrastrain peptide validation S5527,Strain ICDS interstrain exclusive peptide validation S5527,Strain ICDS intrastrain evidence S507,Strain ICDS interstrain exclusive evidence S507,Strain ICDS intrastrain peptide validation S507,Strain ICDS interstrain exclusive peptide validation S507,Reference entries mapped - all strains,ORF ids - all strains
0,S5527,NaN,S5527_scaffold8_size164766|S5527_scaffold8_siz...,S5527_scaffold8_size164766|S5527_scaffold8_siz...,+,-,NaN,NaN,NaN,NaN,P96901,S5527_scaffold8_size164766|S5527_scaffold8_siz...
1,S5527,NaN,S5527_scaffold4_size253745|S5527_scaffold4_siz...,S5527_scaffold4_size253745|S5527_scaffold4_siz...,-,-,NaN,NaN,NaN,NaN,P96202,S5527_scaffold4_size253745|S5527_scaffold4_siz...


In [43]:
interstrain_icds[interstrain_icds['Strain ICDS interstrain exclusive evidence S5527'].notnull()].values[0] 

array(['S5527', nan,
       'S5527_scaffold8_size164766|S5527_scaffold8_size164766_recno_3805.0|(-)38425:39180 and S5527_scaffold8_size164766|S5527_scaffold8_size164766_recno_5843.0|(-)34640:38587 did not align to each other, but both aligned to S507_scaffold11_size165255|S507_scaffold11_size165255_recno_853.0|(+)125359:129900 with an evalue cutoff or 0.0001\n***Frameshift validation peptide FLSGQSPTTIVAPPAAK***\nFLSGQSPTTIVAP matched at aa postion 220-232 in S5527_scaffold8_size164766|S5527_scaffold8_size164766_recno_3805.0|(-)38425:39180 (genomic coordinates: (-)38485:38523 in contig S5527_scaffold8_size164766)\nPAAK matched at aa postion 35-38 in S5527_scaffold8_size164766|S5527_scaffold8_size164766_recno_5843.0|(-)34640:38587 (genomic coordinates: (-)38474:38485 in contig S5527_scaffold8_size164766)',
       'S5527_scaffold8_size164766|S5527_scaffold8_size164766_recno_3805.0|(-)38425:39180 and S5527_scaffold8_size164766|S5527_scaffold8_size164766_recno_5843.0|(-)34640:38587 did not

In [44]:
interstrain_icds[interstrain_icds['Strain ICDS interstrain exclusive evidence S5527'].notnull()].values[1] 

array(['S5527', nan,
       'S5527_scaffold4_size253745|S5527_scaffold4_size253745_recno_3860.0|(+)129483:132575 and S5527_scaffold4_size253745|S5527_scaffold4_size253745_recno_2206.0|(+)131723:136162 did not align to each other, but both aligned to S507_scaffold2_size353395|S507_scaffold2_size353395_recno_10206.0|(-)117515:124195 with an evalue cutoff or 0.0001',
       'S5527_scaffold4_size253745|S5527_scaffold4_size253745_recno_3860.0|(+)129483:132575 and S5527_scaffold4_size253745|S5527_scaffold4_size253745_recno_2206.0|(+)131723:136162 did not align to each other, but both aligned to S507_scaffold2_size353395|S507_scaffold2_size353395_recno_10206.0|(-)117515:124195 with an evalue cutoff or 0.0001',
       '-', '-', nan, nan, nan, nan, 'P96202',
       'S5527_scaffold4_size253745|S5527_scaffold4_size253745_recno_2206.0|(+)131723:136162\nS5527_scaffold4_size253745|S5527_scaffold4_size253745_recno_3860.0|(+)129483:132575\nS507_scaffold2_size353395|S507_scaffold2_size353395_recno_1020

# 3.2.12 ICDSs relative to H37Rv and CDC1551

In [45]:
reference_icds['Reference ICDS evidence'].values[0]  # https://academic.oup.com/gbe/article/5/10/1849/521948 - validated

'tr|P96248|P96248_MYCTU and tr|O07794|O07794_MYCTU did not align to each other, but both aligned to S507_scaffold1_size371796|S507_scaffold1_size371796_recno_109.0|(+)13408:15642;S5527_scaffold9_size227503|S5527_scaffold9_size227503_recno_5032.0|(-)116895:119129 with an evalue cutoff or 0.0001'

In [46]:
reference_icds['Reference ICDS evidence'].values[1]  # https://bmcecolevol.biomedcentral.com/articles/10.1186/1471-2148-8-78

'sp|P9WKC5|TGS3_MYCTU and tr|O05878|O05878_MYCTU did not align to each other, but both aligned to S5527_scaffold8_size164766|S5527_scaffold8_size164766_recno_1618.0|(+)104576:106039;S507_scaffold11_size165255|S507_scaffold11_size165255_recno_3671.0|(-)58499:59962 with an evalue cutoff or 0.0001'

In [47]:
reference_icds['Reference ICDS evidence'].values[2] # https://pubmed.ncbi.nlm.nih.gov/35307510/ 

'sp|O07718|ACEAA_MYCTU and sp|O07717|ACEAB_MYCTU did not align to each other, but both aligned to S507_scaffold14_size153262|S507_scaffold14_size153262_recno_2782.0|(+)130173:132500;S5527_scaffold7_size169925|S5527_scaffold7_size169925_recno_4340.0|(-)4977:7304 with an evalue cutoff or 0.0001'

In [48]:
reference_icds['Reference ICDS evidence'].values[3] # deletion event causing fusion in M Bovis. https://mycobrowser.epfl.ch/genes/Mb2999c

'tr|P95120|P95120_MYCTU and tr|I6Y259|I6Y259_MYCTU did not align to each other, but both aligned to S507_scaffold2_size353395|S507_scaffold2_size353395_recno_2362.0|(+)48095:49810;S5527_scaffold4_size253745|S5527_scaffold4_size253745_recno_6313.0|(-)203867:205582 with an evalue cutoff or 0.0001'

In [49]:
# ICDS relative to H37Rv in either strain
reference_icds

,ICDS,Reference ICDS evidence,Strain ICDS intrastrain evidence S5527,Strain ICDS interstrain exclusive evidence S5527,Strain ICDS intrastrain peptide validation S5527,Strain ICDS interstrain exclusive peptide validation S5527,Strain ICDS intrastrain evidence S507,Strain ICDS interstrain exclusive evidence S507,Strain ICDS intrastrain peptide validation S507,Strain ICDS interstrain exclusive peptide validation S507,Reference entries mapped - all strains,ORF ids - all strains
0,NaN,tr|P96248|P96248_MYCTU and tr|O07794|O07794_MY...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O07794;P96248,S5527_scaffold9_size227503|S5527_scaffold9_siz...
1,NaN,sp|P9WKC5|TGS3_MYCTU and tr|O05878|O05878_MYCT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P9WKC5;O05878,S5527_scaffold8_size164766|S5527_scaffold8_siz...
2,NaN,sp|O07718|ACEAA_MYCTU and sp|O07717|ACEAB_MYCT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O07717;O07718,S5527_scaffold7_size169925|S5527_scaffold7_siz...
3,NaN,tr|P95120|P95120_MYCTU and tr|I6Y259|I6Y259_MY...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I6Y259;P95120,S5527_scaffold4_size253745|S5527_scaffold4_siz...
4,NaN,sp|P96284|PKS15_MYCTU and sp|P96285|PKS1_MYCTU...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P96284;P96285,S5527_scaffold4_size253745|S5527_scaffold4_siz...
5,NaN,tr|O05447|O05447_MYCTU and tr|O07036|O07036_MY...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O07036;O05447,S5527_scaffold9_size227503|S5527_scaffold9_siz...
6,NaN,tr|P71835|P71835_MYCTU and tr|P71834|P71834_MY...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P71835;P71834,S5527_scaffold27_size39624|S5527_scaffold27_si...
7,NaN,sp|P9WFD7|Y2623_MYCTU and sp|P9WL65|Y2628_MYCT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P9WL65;P9WFD7,S5527_scaffold12_size149119|S5527_scaffold12_s...
8,NaN,sp|L0TBY6|2250A_MYCTU and tr|L0TBR2|L0TBR2_MYC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L0TBR2;L0TBY6,S5527_scaffold5_size225283|S5527_scaffold5_siz...
9,S507,sp|Q79FP0|PG29_MYCTU and tr|O53158|O53158_MYCT...,NaN,NaN,NaN,NaN,S507_scaffold19_size115010|S507_scaffold19_siz...,S507_scaffold19_size115010|S507_scaffold19_siz...,-,-,O53158;Q79FP0,S5527_scaffold19_size115875|S5527_scaffold19_s...


## ICDS relative to CDC1551

In [50]:
print(len(reference_icds2)) # cdc1551

9


In [51]:
len(set(reference_icds2['ORF ids - all strains'].tolist()) & set(reference_icds['ORF ids - all strains'].tolist() )) # number of ICDSs shared

7

In [52]:
len(set(reference_icds2['ORF ids - all strains'].tolist()) - set(reference_icds['ORF ids - all strains'].tolist() )) # number of ICDSs exlusive to CDC1551

2

In [53]:
set(reference_icds2['ORF ids - all strains'].tolist()) - set(reference_icds['ORF ids - all strains'].tolist() )

{'S5527_scaffold2_size528256|S5527_scaffold2_size528256_recno_18899.0|(-)5361:6002\nS5527_scaffold34_size12122|S5527_scaffold34_size12122_recno_320.0|(-)10217:10462\nS507_scaffold5_size229288|S507_scaffold5_size229288_recno_3903.0|(+)224616:225257\nS507_scaffold36_size14371|S507_scaffold36_size14371_recno_445.0|(-)9537:9782',
 'S5527_scaffold2_size528256|S5527_scaffold2_size528256_recno_7878.0|(+)248202:249833\nS507_scaffold4_size362088|S507_scaffold4_size362088_recno_11420.0|(-)344043:345674'}

In [54]:
len(set(reference_icds['ORF ids - all strains'].tolist()) - set(reference_icds2['ORF ids - all strains'].tolist() )) # number of ICDSs exlusive to H37Rv

3

In [55]:
reference_icds_filt2 = fs2[['Reference ICDS evidence', 'ORF ids - all strains','Reference entries mapped - all strains','Strain ICDS interstrain exclusive evidence S5527','Strain ICDS interstrain exclusive evidence S507']]
reference_icds_filt2.rename(columns={"Reference ICDS evidence" : "CDC1551 ICDS Evidence",'Reference entries mapped - all strains':'CDC1551 Reference Mapped'}, inplace=True)
reference_icds_filt2

reference_icds_filt = fs[['Reference ICDS evidence', 'ORF ids - all strains','Reference entries mapped - all strains','Strain ICDS interstrain exclusive evidence S5527','Strain ICDS intrastrain evidence S507']]
reference_icds_filt.rename(columns={"Reference ICDS evidence" : "H37Rv ICDS Evidence",'Reference entries mapped - all strains':'H37Rv Reference Mapped'}, inplace=True)
reference_icds_filt
combined_icds =pd.merge(reference_icds_filt, reference_icds_filt2,how="outer")
combined_icds = combined_icds[["H37Rv ICDS Evidence"	,'H37Rv Reference Mapped', "CDC1551 ICDS Evidence",'CDC1551 Reference Mapped', "ORF ids - all strains",'Strain ICDS interstrain exclusive evidence S5527','Strain ICDS interstrain exclusive evidence S507']]

def simplify_ref(x):
    try:
        _ = x.split(' and ')
    except:
        return
    a = _[0].split('|')[1]
    b = _[1].split('|')[1]
    return a +  ' and ' + b


def simplify_orfs(x):
    x = x.split('\n')
    new = [i.split('|')[1] for i in x]
    return ';'.join(new)
    
combined_icds['H37Rv ICDS Evidence'] = combined_icds['H37Rv ICDS Evidence'].apply(simplify_ref)
combined_icds['CDC1551 ICDS Evidence'] = combined_icds['CDC1551 ICDS Evidence'].apply(simplify_ref)
combined_icds['ORF ids - all strains'] = combined_icds['ORF ids - all strains'].apply(simplify_orfs)

def strain_icds(x):
    try: 
        x = x.split('\n')
        _ = x[0].split(' and ')
        first = _[0].split('|')[1]
        second = _[1].split('|')[1].split()[0]
        ls = [first, second]
        x = ' and '.join([first, second])
    except:
        return
    return x

combined_icds['Strain ICDS interstrain exclusive evidence S507'] =combined_icds['Strain ICDS interstrain exclusive evidence S507'].apply(strain_icds)
combined_icds['Strain ICDS interstrain exclusive evidence S5527'] =combined_icds['Strain ICDS interstrain exclusive evidence S5527'].apply(strain_icds)
combined_icds.rename(columns = {'Strain ICDS interstrain exclusive evidence S507':'Beijing (S507) ICDS Evidence','Strain ICDS interstrain exclusive evidence S5527':'Beijing (S5527) ICDS Evidence'},inplace=True)

col_order = ['H37Rv ICDS Evidence', 'CDC1551 ICDS Evidence',	'Beijing (S5527) ICDS Evidence',	'Beijing (S507) ICDS Evidence', 'H37Rv Reference Mapped',		'CDC1551 Reference Mapped',	'ORF ids - all strains']
combined_icds = combined_icds[col_order]
combined_icds.to_excel('supplementary/s14.xlsx')

combined_icds

<ipython-input-55-5fb97e0b7236>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reference_icds_filt2.rename(columns={"Reference ICDS evidence" : "CDC1551 ICDS Evidence",'Reference entries mapped - all strains':'CDC1551 Reference Mapped'}, inplace=True)
<ipython-input-55-5fb97e0b7236>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reference_icds_filt.rename(columns={"Reference ICDS evidence" : "H37Rv ICDS Evidence",'Reference entries mapped - all strains':'H37Rv Reference Mapped'}, inplace=True)


,H37Rv ICDS Evidence,CDC1551 ICDS Evidence,Beijing (S5527) ICDS Evidence,Beijing (S507) ICDS Evidence,H37Rv Reference Mapped,CDC1551 Reference Mapped,ORF ids - all strains
0,P96248 and O07794,L7N5U9 and L7N4W6,None,None,O07794;P96248,L7N5U9;L7N4W6,S5527_scaffold9_size227503_recno_5032.0;S507_s...
1,P9WKC5 and O05878,P9WKC4 and Q7D5V7,None,None,P9WKC5;O05878,Q7D5V7;P9WKC4,S5527_scaffold8_size164766_recno_1618.0;S507_s...
2,None,None,S5527_scaffold8_size164766_recno_3805.0 and S5...,None,P96901,L7N527,S5527_scaffold8_size164766_recno_5843.0;S5527_...
3,O07718 and O07717,None,None,None,O07717;O07718,NaN,S5527_scaffold7_size169925_recno_4340.0;S507_s...
4,P95120 and I6Y259,Q7D6B8 and P95121,None,None,I6Y259;P95120,Q7D6B8;P95121,S5527_scaffold4_size253745_recno_6313.0;S507_s...
5,P96284 and P96285,Q7D6E0 and Q7D6E1,None,None,P96284;P96285,Q7D6E1;Q7D6E0,S5527_scaffold4_size253745_recno_4951.0;S507_s...
6,O05447 and O07036,L7N567 and Q8VIR3,None,None,O07036;O05447,Q8VIR3;L7N567,S5527_scaffold9_size227503_recno_4531.0;S507_s...
7,None,None,S5527_scaffold4_size253745_recno_3860.0 and S5...,None,P96202,Q7D6F1,S5527_scaffold4_size253745_recno_2206.0;S5527_...
8,P71835 and P71834,None,None,None,P71835;P71834,NaN,S5527_scaffold27_size39624_recno_890.0;S507_sc...
9,P9WFD7 and P9WL65,P9WFD6 and P9WL64,None,None,P9WL65;P9WFD7,P9WL64;P9WFD6,S5527_scaffold12_size149119_recno_2406.0;S507_...


## 3.2.13 Gene model validations

In [56]:
annotations = pd.read_csv("data/analysis/annotations/annotations.csv")
annotations.rename(columns={"Unnamed: 0":''},inplace=True)
annotations.to_excel('tables/table4.xlsx', index=False)
annotations

,,CDC1551,H37Rv
0,Reference entries identified,2065,2098
1,Annotated TSS validated,434,474
2,Downstream TSS identified,397,378
3,Upstream non-TSS peptide with no putative upst...,93,94
4,Upstream TSS identified,56,51
5,Upstream non-TSS peptide with putative upstrea...,82,68


In [57]:
cdc1551_validations = pd.read_csv("data/analysis/annotations/export/CDC1551/Combined_Annotated_TSS_validated.csv")
print('CDC1551 gene model validations: ', cdc1551_validations['AnnotationId'].max())

h37rv_validations = pd.read_csv("data/analysis/annotations/export/H37Rv/Combined_Annotated_TSS_validated.csv")
print('H37Rv gene model validations: ', h37rv_validations['AnnotationId'].max())

CDC1551 gene model validations:  434
H37Rv gene model validations:  474


In [58]:
h37rv_validations[h37rv_validations['BLASTP'] == 'P9WK87']

,Unnamed: 0,ORF_id,Strain_Nterm_Acetylated,TSS,Enzymatic,StartCodon,StartPosition,PeptidePosition,StartType,AnnotationMessage,...,Identified S507_ML,Acetylated S507_ML,Identified S507_MLexp,Acetylated S507_MLexp,Identified S5527_ML,Acetylated S5527_ML,Cluster,Strain,ORF_start_count,AnnotationId
327,15085,S5527_scaffold19_size115875|S5527_scaffold19_s...,True,True,False,ATG,24.0,9,MAP-cleaved iMet,Non-tryptic N-terminus peptide TEPTVARPDIDPVLK...,...,False,False,False,False,True,True,P9WK87_9,S5527,1.0,155


## 3.2.14 Downstream gene model reannotations

In [59]:
cdc1551_downstream = pd.read_csv("data/analysis/annotations/export/CDC1551/Combined_Downstream_TSS_identified.csv")
print('CDC1551 downstream gene model modifications: ', cdc1551_downstream['AnnotationId'].max())

h37rv_downstream = pd.read_csv("data/analysis/annotations/export/H37Rv/Combined_Downstream_TSS_identified.csv")
print('H37Rv downstream gene model modifications: ', h37rv_downstream['AnnotationId'].max())

CDC1551 downstream gene model modifications:  397
H37Rv downstream gene model modifications:  378


In [60]:
h37rv_downstream[h37rv_downstream['Ref_id'] == 'P9WMK7']

,Unnamed: 0,ORF_id,Strain_Nterm_Acetylated,TSS,Enzymatic,StartCodon,StartPosition,PeptidePosition,StartType,AnnotationMessage,...,Identified S507_ML,Acetylated S507_ML,Identified S507_MLexp,Acetylated S507_MLexp,Identified S5527_ML,Acetylated S5527_ML,Cluster,Strain,ORF_start_count,AnnotationId
123,17310,S5527_scaffold4_size253745|S5527_scaffold4_siz...,False,True,False,ATG,39.0,14,MAP-cleaved iMet,Non-tryptic N-terminus peptide NKAELIDVLTQK fo...,...,True,False,True,False,True,False,P9WMK7_14,S5527,1.0,76
124,17302,S5527_scaffold4_size253745|S5527_scaffold4_siz...,True,True,False,ATG,39.0,13,N-terminal acetylated iMet,N-terminal acetylated iMet peptide MNKAELIDVLT...,...,True,False,True,False,True,True,P9WMK7_13,S5527,1.0,76
125,17193,S507_scaffold2_size353395|S507_scaffold2_size3...,False,True,False,ATG,39.0,14,MAP-cleaved iMet,Non-tryptic N-terminus peptide NKAELIDVLTQK fo...,...,True,False,True,False,True,False,P9WMK7_14,S507,1.0,76
126,17185,S507_scaffold2_size353395|S507_scaffold2_size3...,False,True,False,ATG,39.0,13,Non-enzymatic iMet,Non-enzymatic iMet peptide MNKAELIDVLTQK ident...,...,True,False,True,False,True,True,P9WMK7_13,S507,1.0,76


## 3.2.15 Upstream gene model reannotations

In [61]:
cdc1551_upstream = pd.read_csv("data/analysis/annotations/export/CDC1551/Combined_Upstream_TSS_identified.csv")
print('CDC1551 upstream gene model modifications: ', cdc1551_upstream['AnnotationId'].max())

h37rv_upstream = pd.read_csv("data/analysis/annotations/export/H37Rv/Combined_Upstream_TSS_identified.csv")
print('H37Rv upstream gene model modifications: ', h37rv_upstream['AnnotationId'].max())

CDC1551 upstream gene model modifications:  56
H37Rv upstream gene model modifications:  51


In [62]:
cdc1551_upstream_putative = pd.read_csv("data/analysis/annotations/export/CDC1551/Combined_Upstream_non-TSS_peptide_with_putative_upstream_TSS.csv")
print('CDC1551 upstream gene model modifications, putative upstream TSS: ', cdc1551_upstream_putative['AnnotationId'].max())

h37rv_upstream_putative= pd.read_csv("data/analysis/annotations/export/H37Rv/Combined_Upstream_non-TSS_peptide_with_putative_upstream_TSS.csv")
print('H37Rv upstream gene model modifications, putative upstream TSS: ', h37rv_upstream_putative['AnnotationId'].max())

CDC1551 upstream gene model modifications, putative upstream TSS:  82
H37Rv upstream gene model modifications, putative upstream TSS:  68


In [63]:
cdc1551_upstream_no_putative = pd.read_csv("data/analysis/annotations/export/CDC1551/Combined_Upstream_non-TSS_peptide_with_no_putative_upstream_TSS.csv")
print('CDC1551 upstream gene model modifications, no putative upstream TSS: ', cdc1551_upstream_no_putative['AnnotationId'].max())

h37rv_upstream_no_putative= pd.read_csv("data/analysis/annotations/export/H37Rv/Combined_Upstream_non-TSS_peptide_with_no_putative_upstream_TSS.csv")
print('H37Rv upstream gene model modifications, no uptative upstream TSS: ', h37rv_upstream_no_putative['AnnotationId'].max())

CDC1551 upstream gene model modifications, no putative upstream TSS:  93
H37Rv upstream gene model modifications, no uptative upstream TSS:  94


## 3.2.16 Novel gene models

In [64]:

cdc1551_novel = pd.read_csv("data/analysis/annotations/export/CDC1551/Combined_Novel_ORFs.csv")

print('CDC1551 novel ORFs: ', cdc1551_novel['AnnotationId'].max())

h37rv_novel = pd.read_csv("data/analysis/annotations/export/H37Rv/Combined_Novel_ORFs.csv")

print('H37Rv novel ORFs: ', h37rv_novel['AnnotationId'].max())

CDC1551 novel ORFs:  72
H37Rv novel ORFs:  32


In [65]:
combined_peptide_annotations = pd.concat([s507_peptide_annotations,s5527_peptide_annotations])

In [66]:
with open('supplementary/s13_20_12_2024.json') as f:
    h37rv_blast = json.loads(f.read())

h37rv_blast_res = {}
for rec in h37rv_blast['BlastOutput2']:
    query=rec['report']['results']['search']['query_title'].split()[0]

    
    hits  = rec['report']['results']['search']['hits']
    if len(hits) == 0:
        continue
    target = rec['report']['results']['search']['hits'][0]

    h37rv_blast_res[query]=target


def count_orf_peptides(orf):
    orf_peptides = combined_peptide_annotations[combined_peptide_annotations['ORF_id'] == orf].drop_duplicates('PeptideSequence')
    return len(orf_peptides)



h37rv_novel_blasted = h37rv_novel.copy()
h37rv_novel_blasted['title']=h37rv_novel_blasted['ORF_id'].apply(lambda x : h37rv_blast_res[x]['description'][0]['title'] if x in h37rv_blast_res else np.nan)
h37rv_novel_blasted['id']=h37rv_novel_blasted['ORF_id'].apply(lambda x : h37rv_blast_res[x]['description'][0]['id'] if x in h37rv_blast_res else np.nan)
h37rv_novel_blasted['accession']=h37rv_novel_blasted['ORF_id'].apply(lambda x : h37rv_blast_res[x]['description'][0]['accession'] if x in h37rv_blast_res else np.nan)
h37rv_novel_blasted['sciname']=h37rv_novel_blasted['ORF_id'].apply(lambda x : h37rv_blast_res[x]['description'][0]['sciname'] if x in h37rv_blast_res else np.nan)
h37rv_novel_blasted['evalue']=h37rv_novel_blasted['ORF_id'].apply(lambda x : h37rv_blast_res[x]['hsps'][0]['evalue'] if x in h37rv_blast_res else np.nan)

h37rv_novel_blasted['SpecificPeptides'] = h37rv_novel_blasted['ORF_id'].apply(lambda x : count_orf_peptides(x))

h37rv_novel_blasted['ORF_id']=h37rv_novel_blasted['ORF_id'].apply(lambda x : x.split('|')[1])
h37rv_novel_blasted.sort_values('accession',inplace=True)

h37rv_novel_blasted.to_excel('supplementary/s14.xlsx')
print('Any number of specific peptides: ',Counter(h37rv_novel_blasted['ORF_id'].apply(lambda x : x.split('_')[0])))
print('Two or more specific peptides: ',Counter(h37rv_novel_blasted[h37rv_novel_blasted['SpecificPeptides']>1]['ORF_id'].apply(lambda x : x.split('_')[0])))

h37rv_novel_blasted[h37rv_novel_blasted['SpecificPeptides']>1]

Any number of specific peptides:  Counter({'S507': 16, 'S5527': 16})
Two or more specific peptides:  Counter({'S507': 6, 'S5527': 6})


,ORF_id,AnnotationId,title,id,accession,sciname,evalue,SpecificPeptides
0,S507_scaffold13_size114854_recno_302.0,1,2-dehydropantoate 2-reductase [Mycobacterium t...,gb|AGL28028.1|,AGL28028,Mycobacterium tuberculosis CAS/NITR204,7.263960e-08,2
17,S5527_scaffold12_size149119_recno_307.0,18,2-dehydropantoate 2-reductase [Mycobacterium t...,gb|AGL28028.1|,AGL28028,Mycobacterium tuberculosis CAS/NITR204,7.263960e-08,2
27,S5527_scaffold4_size253745_recno_4408.0,28,hypothetical protein CAB90_03371 [Mycobacteriu...,gb|AUS52195.1|,AUS52195,Mycobacterium tuberculosis,5.951260e-133,3
9,S507_scaffold2_size353395_recno_12947.0,10,hypothetical protein CAB90_03371 [Mycobacteriu...,gb|AUS52195.1|,AUS52195,Mycobacterium tuberculosis,5.951260e-133,2
31,S5527_scaffold7_size169925_recno_6453.0,32,Uncharacterised protein [Mycobacterium tubercu...,emb|CFR88921.1|,CFR88921,Mycobacterium tuberculosis,0.000000e+00,2
1,S507_scaffold14_size153262_recno_1808.0,2,Uncharacterised protein [Mycobacterium tubercu...,emb|CFR88921.1|,CFR88921,Mycobacterium tuberculosis,0.000000e+00,2
28,S5527_scaffold4_size253745_recno_6544.0,29,polyketide synthase [Mycobacterium tuberculosis],emb|CKM60639.1|,CKM60639,Mycobacterium tuberculosis,1.406410e-16,2
10,S507_scaffold2_size353395_recno_2593.0,11,polyketide synthase [Mycobacterium tuberculosis],emb|CKM60639.1|,CKM60639,Mycobacterium tuberculosis,1.406410e-16,2
20,S5527_scaffold19_size115875_recno_2514.0,21,"LOW QUALITY PROTEIN: lipase lipH, partial [Myc...",gb|EFD57973.1|,EFD57973,Mycobacterium tuberculosis T92,4.982640e-94,2
19,S5527_scaffold18_size93797_recno_3132.0,20,MULTISPECIES: hypothetical protein [Mycobacter...,ref|WP_003911400.1|,WP_003911400,Mycobacterium,2.204200e-39,2


## 3.2.17 Potential gene duplication events

In [67]:

filt_annot = (combined_peptide_annotations
              [combined_peptide_annotations['Ref_id'].notnull()]
              .drop_duplicates(['Ref_id', 'ORF_id'], keep='first'))

filt_annot['Strain'] = filt_annot['ORF_id'].apply(lambda x: x.split('_')[0])


counts_df = pd.crosstab(index=filt_annot['Ref_id'], 
                        columns=filt_annot['Strain'])
counts_df = counts_df.loc[counts_df.sum(axis=1).sort_values(ascending=False).index]

counts_df= counts_df[~((counts_df['S507'] <=1) & (counts_df['S5527'] <=1))]

print(len(counts_df[counts_df['S507'] > counts_df['S5527'] ]))
print(len(counts_df[counts_df['S507'] < counts_df['S5527'] ]))
print(len(counts_df))
counts_df.to_excel('supplementary/s15.xlsx')
counts_df

6
16
31


Strain,S507,S5527
Ref_id,,
P9WNI5,4,4
P9WNJ7,4,4
O07182,2,5
O53377,2,5
I6XD69,4,3
P9WNJ5,3,2
P9WQE7,3,2
I6X5G8,2,2
P9WK81,2,2


In [68]:

combined_peptide_annotations[combined_peptide_annotations['Ref_id'] =='P9WNI5']['ORF_id'].unique()

array(['S507_scaffold6_size207594|S507_scaffold6_size207594_recno_58.0|(+)7840:8181',
       'S507_scaffold8_size205631|S507_scaffold8_size205631_recno_4910.0|(-)540:881',
       'S507_scaffold14_size153262|S507_scaffold14_size153262_recno_2.0|(+)34:375',
       'S507_scaffold50_size1304|S507_scaffold50_size1304_recno_33.0|(-)930:1271',
       'S5527_scaffold21_size68727|S5527_scaffold21_size68727_recno_1629.0|(-)415:756',
       'S5527_scaffold28_size35063|S5527_scaffold28_size35063_recno_898.0|(-)34688:34966',
       'S5527_scaffold37_size8979|S5527_scaffold37_size8979_recno_173.0|(+)8334:8675',
       'S5527_scaffold57_size3198|S5527_scaffold57_size3198_recno_27.0|(+)1688:2029'],
      dtype=object)

In [69]:
combined_peptide_annotations[combined_peptide_annotations['Ref_id'] =='P9WNJ7']['ORF_id'].unique()

array(['S507_scaffold6_size207594|S507_scaffold6_size207594_recno_58.0|(+)7840:8181',
       'S507_scaffold8_size205631|S507_scaffold8_size205631_recno_4910.0|(-)540:881',
       'S507_scaffold14_size153262|S507_scaffold14_size153262_recno_2.0|(+)34:375',
       'S507_scaffold50_size1304|S507_scaffold50_size1304_recno_33.0|(-)930:1271',
       'S5527_scaffold21_size68727|S5527_scaffold21_size68727_recno_1629.0|(-)415:756',
       'S5527_scaffold28_size35063|S5527_scaffold28_size35063_recno_898.0|(-)34688:34966',
       'S5527_scaffold37_size8979|S5527_scaffold37_size8979_recno_173.0|(+)8334:8675',
       'S5527_scaffold57_size3198|S5527_scaffold57_size3198_recno_27.0|(+)1688:2029'],
      dtype=object)

In [70]:
combined_peptide_annotations[combined_peptide_annotations['Ref_id'] =='O07182']['ORF_id'].unique()

array(['S507_scaffold29_size43595|S507_scaffold29_size43595_recno_1026.0|(-)40988:42538',
       'S507_scaffold35_size20576|S507_scaffold35_size20576_recno_1.0|(+)1:744',
       'S5527_scaffold33_size20007|S5527_scaffold33_size20007_recno_96.0|(+)395:1945',
       'S5527_scaffold31_size21391|S5527_scaffold31_size21391_recno_601.0|(-)20475:21389',
       'S5527_scaffold55_size1682|S5527_scaffold55_size1682_recno_8.0|(+)937:1680',
       'S5527_scaffold62_size1491|S5527_scaffold62_size1491_recno_58.0|(-)3:227',
       'S5527_scaffold64_size1458|S5527_scaffold64_size1458_recno_51.0|(-)3:584'],
      dtype=object)

In [71]:
combined_peptide_annotations[combined_peptide_annotations['Ref_id'] =='O53377']['ORF_id'].unique()

array(['S507_scaffold29_size43595|S507_scaffold29_size43595_recno_1026.0|(-)40988:42538',
       'S507_scaffold35_size20576|S507_scaffold35_size20576_recno_1.0|(+)1:744',
       'S5527_scaffold33_size20007|S5527_scaffold33_size20007_recno_96.0|(+)395:1945',
       'S5527_scaffold31_size21391|S5527_scaffold31_size21391_recno_601.0|(-)20475:21389',
       'S5527_scaffold55_size1682|S5527_scaffold55_size1682_recno_8.0|(+)937:1680',
       'S5527_scaffold62_size1491|S5527_scaffold62_size1491_recno_58.0|(-)3:227',
       'S5527_scaffold64_size1458|S5527_scaffold64_size1458_recno_51.0|(-)3:584'],
      dtype=object)

In [72]:
combined[combined[hypercols].isnull().all(axis=1)]

NameError: name 'hypercols' is not defined

In [ ]:
exp_cols = [col for col in combined.columns if col.startswith('Experiment ')]

In [ ]:
exp_cols

In [ ]:
combined.columns.tolist()

In [ ]:
combined[combined[['All peptides group S507MLexp', 'All peptides group S507ML', 'All peptides group S507ST']].isnull().all(axis=1)].stack()

In [ ]:
combined[combined[['All peptides group S5527MLexp', 'All peptides group S5527ML', 'All peptides group S5527ST']].isnull().all(axis=1)].stack()

In [ ]:
temp = pd.read_csv('/cbio/users/ptgmat003/2023/Thys/mtb/manuscripts/proteogenomics/Proteogenomics-Manuscript/data/analysis/strains/S507/S507_mapped_peptides.csv')

In [ ]:
temp[(temp['ORF_id'].str.contains('S507_scaffold1_size371796_recno_2394.0')) & (temp["Strain_identified"]=='-')]['Peptide_sequence']

In [ ]:
s5527_temp = pd.read_csv('/cbio/users/ptgmat003/2023/Thys/mtb/manuscripts/proteogenomics/Proteogenomics-Manuscript/data/analysis/strains/S5527/S5527_mapped_peptides.csv')

In [ ]:
s5527_temp[s5527_temp['Peptide_sequence']=='YGQQVAAVVQAR']['ORF_id'].values

In [ ]:
temp[temp['Peptide_sequence']=='SEIAGYKVPR']['ORF_id'].values

In [ ]:
temp